# Image captioning with visual attention using Tensorflow 2.0

### Goal : Build an attention-based model, which enables us to see what parts of the image the model focuses on as it generates a caption.

### The dataset that will be used for this tutorial is the [COCO Dataset](http://cocodataset.org/#home)

![ImageCaptioning](https://cdn.analyticsvidhya.com/wp-content/uploads/2018/03/example.png)

#### We train about 30,000 captions for about 20,000 images here
#### (Later we should expand for better accuracy)

In [47]:
import tensorflow as tf
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import platform
import datetime
import os
import math
import random
import pathlib
import json
from glob import glob
from PIL import Image
import pickle
import re
from tqdm import tqdm
import time

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tensorboard

Python version: 3.7.6
Tensorflow version: 2.2.0
Keras version: 2.3.0-tf
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [8]:
# checking current directory
os.path.abspath('./data')

'C:\\Users\\bokhy\\Desktop\\Python\\Python-Projects\\Tensorflow\\data'

# 1. Load Data
##### https://www.tensorflow.org/datasets/overview
##### http://cocodataset.org/#download (we get the 2014 version)

In [9]:
# The dataset has > 80,000 images, each of which has at least 5 different annotations

# Download annotation files
annotation_folder = '/annotations/'
if not os.path.exists(os.path.abspath('./data') + annotation_folder):
    annotation_zip = tf.keras.utils.get_file('captions.zip',
                                              cache_subdir=os.path.abspath('.'),
                                              origin = 'http://images.cocodataset.org/annotations/annotations_trainval2014.zip',
                                              extract = True)
    annotation_file = os.path.dirname(annotation_zip)+'/annotations/captions_train2014.json'
    os.remove(annotation_zip)

# Download image files
image_folder = '/train2014/'
if not os.path.exists(os.path.abspath('./data') + image_folder):
    image_zip = tf.keras.utils.get_file('train2014.zip',
                                          cache_subdir=os.path.abspath('.'),
                                          origin = 'http://images.cocodataset.org/zips/train2014.zip',
                                          extract = True)
    PATH = os.path.dirname(image_zip) + image_folder
    os.remove(image_zip)
else:
    PATH = os.path.abspath('.') + image_folder

13510574080/13510573713 [==============================] - 2383s 0us/step


##### To Save time, let's use around 30,000 captions and their corresponding images to train our model. 
##### ToDo: Increase the data size to improve quality

In [10]:
# Read the json file
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

# Store captions and image names in vectors
all_captions = []
all_img_name_vector = []

for annot in annotations['annotations']:
    caption = '<start> ' + annot['caption'] + ' <end>'
    image_id = annot['image_id']
    full_coco_image_path = PATH + 'COCO_train2014_' + '%012d.jpg' % (image_id)

    all_img_name_vector.append(full_coco_image_path)
    all_captions.append(caption)

# Shuffle captions and image_names together
# Set a random state
train_captions, img_name_vector = shuffle(all_captions,
                                          all_img_name_vector,
                                          random_state=1)

# Select the first 30000 captions from the shuffled set
num_examples = 30000
train_captions = train_captions[:num_examples]
img_name_vector = img_name_vector[:num_examples]

In [11]:
len(train_captions), len(all_captions)

(30000, 414113)

# 2. Load the image

In [18]:
def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    # Resizing to 299 x 299 pixel
    img = tf.image.resize(img, (299, 299))
    # Preprocess the images using InceptionV3
    img = tf.keras.applications.inception_v3.preprocess_input(img)
    return img, image_path

# 3. Initialize InceptionV3 and load the pretrained Imagenet weights

In [19]:
image_model = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet')
new_input = image_model.input

# Now create a model where the output layer is the last convolutional layer in the InceptionV3 architecture.
hidden_layer = image_model.layers[-1].output

image_features_extract_model = tf.keras.Model(inputs = new_input,  # base model
                                              outputs = hidden_layer) # last layers

In [20]:
hidden_layer

<tf.Tensor 'mixed10_1/Identity:0' shape=(None, None, None, 2048) dtype=float32>

In [22]:
image_features_extract_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, None, None, 3 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, None, None, 3 0           batch_normalization_94[0][0]     
____________________________________________________________________________________________

# 4. Get features extracted from InceptionV3
#### This step takes some time ###

In [23]:
# Get unique images
encode_train = sorted(set(img_name_vector))

BATCH_SIZE = 16

# Feel free to change batch_size according to your system configuration
image_dataset = tf.data.Dataset.from_tensor_slices(encode_train)
image_dataset = image_dataset.map(
  load_image, num_parallel_calls=tf.data.experimental.AUTOTUNE).batch(batch_size = BATCH_SIZE)

for img, path in image_dataset:
    batch_features = image_features_extract_model(img)
    batch_features = tf.reshape(batch_features,
                                  (batch_features.shape[0], -1, batch_features.shape[3]))

    for bf, p in zip(batch_features, path):
        path_of_feature = p.numpy().decode("utf-8")
        np.save(path_of_feature, bf.numpy())

# 5. Tokenize the captions

> First, you'll tokenize the captions (for example, by splitting on spaces). This gives us a vocabulary of all of the unique words in the data (for example, "surfing", "football", and so on).

> Next, you'll limit the vocabulary size to the top 5,000 words (to save memory). You'll replace all other words with the token "UNK" (unknown).

> You then create word-to-index and index-to-word mappings.

> Finally, you pad all sequences to be the same length as the longest one.

In [24]:
# Find the maximum length of any caption in our dataset
def calc_max_length(tensor):
    return max(len(t) for t in tensor)

In [25]:
# Choose the top 5000 words from the vocabulary
top_k = 5000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=top_k,
                                                  oov_token="<unk>",
                                                  filters='!"#$%&()*+.,-/:;=?@[\]^_`{|}~ ')
tokenizer.fit_on_texts(train_captions)
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [26]:
tokenizer.word_index['<pad>'] = 0
tokenizer.index_word[0] = '<pad>'

In [27]:
# Create the tokenized vectors
train_seqs = tokenizer.texts_to_sequences(train_captions)

In [28]:
# Pad each vector to the max_length of the captions
# If you do not provide a max_length value, pad_sequences calculates it automatically
cap_vector = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding='post')

In [31]:
cap_vector

array([[   3,    2,  351, ...,    0,    0,    0],
       [   3,    2,   31, ...,    0,    0,    0],
       [   3,    2,  318, ...,    0,    0,    0],
       ...,
       [   3,    2,  201, ...,    0,    0,    0],
       [   3, 1462,  264, ...,    0,    0,    0],
       [   3,    2,  175, ...,    0,    0,    0]])

In [29]:
# Calculates the max_length, which is used to store the attention weights
max_length = calc_max_length(train_seqs)

In [30]:
max_length

49

# 6. Train/Test Split

In [32]:
# Create training and validation sets using an 80-20 split
img_name_train, img_name_val, cap_train, cap_val = train_test_split(img_name_vector,
                                                                    cap_vector,
                                                                    test_size=0.2,
                                                                    random_state=623)

In [33]:
len(img_name_train), len(cap_train), len(img_name_val), len(cap_val)

(24000, 24000, 6000, 6000)

# 7. Create a Model

In [34]:
# Feel free to change these parameters according to your system's configuration

BATCH_SIZE = 64
BUFFER_SIZE = 1000
embedding_dim = 256
units = 512
vocab_size = top_k + 1
num_steps = len(img_name_train) // BATCH_SIZE

# Shape of the vector extracted from InceptionV3 is (64, 2048)
features_shape = 2048
attention_features_shape = 64

In [35]:
# Load the numpy files
def map_func(img_name, cap):
    img_tensor = np.load(img_name.decode('utf-8')+'.npy')
    return img_tensor, cap

In [36]:
dataset = tf.data.Dataset.from_tensor_slices((img_name_train, cap_train))

# Use map to load the numpy files in parallel
dataset = dataset.map(lambda item1, item2: tf.numpy_function(
          map_func, [item1, item2], [tf.float32, tf.int32]),
          num_parallel_calls=tf.data.experimental.AUTOTUNE)

# Shuffle and batch
dataset = dataset.shuffle(buffer_size = BUFFER_SIZE).batch(batch_size = BATCH_SIZE)
dataset = dataset.prefetch(buffer_size = tf.data.experimental.AUTOTUNE)

print(dataset)

<PrefetchDataset shapes: (<unknown>, <unknown>), types: (tf.float32, tf.int32)>


### Architecture: https://arxiv.org/pdf/1502.03044.pdf

> In this example, you extract the features from the lower convolutional layer of InceptionV3 giving us a vector of shape (8, 8, 2048).

> Squash that to a shape of (64, 2048).

> This vector is then passed through the CNN Encoder (which consists of a single Fully connected layer).

> The RNN (here GRU) attends over the image to predict the next word

In [37]:
class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, features, hidden):
        # features(CNN_encoder output) shape == (batch_size, 64, embedding_dim)

        # hidden shape == (batch_size, hidden_size)
        # hidden_with_time_axis shape == (batch_size, 1, hidden_size)
        hidden_with_time_axis = tf.expand_dims(hidden, 1)

        # score shape == (batch_size, 64, hidden_size)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis))

        # attention_weights shape == (batch_size, 64, 1)
        # you get 1 at the last axis because you are applying score to self.V
        attention_weights = tf.nn.softmax(self.V(score), axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [38]:
class CNN_Encoder(tf.keras.Model):
    # Since you have already extracted the features and dumped it using pickle
    # This encoder passes those features through a Fully connected layer
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        # shape after fc == (batch_size, 64, embedding_dim)
        self.fc = tf.keras.layers.Dense(embedding_dim)

    def call(self, x):
        x = self.fc(x)
        x = tf.nn.relu(x)
        return x

In [39]:
class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)

        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        # defining attention as a separate model
        context_vector, attention_weights = self.attention(features, hidden)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU
        output, state = self.gru(x)

        # shape == (batch_size, max_length, hidden_size)
        x = self.fc1(output)

        # x shape == (batch_size * max_length, hidden_size)
        x = tf.reshape(x, (-1, x.shape[2]))

        # output shape == (batch_size * max_length, vocab)
        x = self.fc2(x)

        return x, state, attention_weights

    def reset_state(self, batch_size):
        return tf.zeros((batch_size, self.units))

In [40]:
encoder = CNN_Encoder(embedding_dim)
decoder = RNN_Decoder(embedding_dim, units, vocab_size)

In [41]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [42]:
# Checkpoint
checkpoint_path = "./checkpoints/train"
ckpt = tf.train.Checkpoint(encoder=encoder,
                           decoder=decoder,
                           optimizer = optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

In [43]:
start_epoch = 0
if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

# 8. Train a Model 

> 1. You extract the features stored in the respective .npy files and then pass those features through the encoder.
> 2. The encoder output, hidden state(initialized to 0) and the decoder input (which is the start token) is passed to the decoder.
> 3. The decoder returns the predictions and the decoder hidden state.
> 4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
> 5. Use teacher forcing to decide the next input to the decoder.
> 6. Teacher forcing is the technique where the target word is passed as the next input to the decoder.
> 7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [44]:
# adding this in a separate cell because if you run the training cell
# many times, the loss_plot array will be reset
loss_plot = []

In [45]:
@tf.function
def train_step(img_tensor, target):
    loss = 0

      # initializing the hidden state for each batch
      # because the captions are not related from image to image
    hidden = decoder.reset_state(batch_size=target.shape[0])

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)

        for i in range(1, target.shape[1]):
            # passing the features through the decoder
            predictions, hidden, _ = decoder(dec_input, features, hidden)

            loss += loss_function(target[:, i], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(target[:, i], 1)

        total_loss = (loss / int(target.shape[1]))

        trainable_variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, trainable_variables)

        optimizer.apply_gradients(zip(gradients, trainable_variables))

        return loss, total_loss

In [48]:
EPOCHS = 15

for epoch in range(start_epoch, EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss, t_loss = train_step(img_tensor, target)
        total_loss += t_loss

        if batch % 100 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(
              epoch + 1, batch, batch_loss.numpy() / int(target.shape[1])))
    # storing the epoch end loss value to plot later
    loss_plot.append(total_loss / num_steps)

    if epoch % 5 == 0:
        ckpt_manager.save()

    print ('Epoch {} Loss {:.6f}'.format(epoch + 1,
                                         total_loss/num_steps))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 2.1113
Epoch 1 Batch 100 Loss 1.0932
Epoch 1 Batch 200 Loss 0.9566
Epoch 1 Batch 300 Loss 0.8957
Epoch 1 Loss 1.038244
Time taken for 1 epoch 1411.6559073925018 sec

Epoch 2 Batch 0 Loss 0.9324
Epoch 2 Batch 100 Loss 0.8369
Epoch 2 Batch 200 Loss 0.8219
Epoch 2 Batch 300 Loss 0.7188
Epoch 2 Loss 0.796382
Time taken for 1 epoch 1307.3182227611542 sec

Epoch 3 Batch 0 Loss 0.8305
Epoch 3 Batch 100 Loss 0.6964
Epoch 3 Batch 200 Loss 0.6661
Epoch 3 Batch 300 Loss 0.7101
Epoch 3 Loss 0.724262
Time taken for 1 epoch 1445.2355513572693 sec

Epoch 4 Batch 0 Loss 0.7114
Epoch 4 Batch 100 Loss 0.7761
Epoch 4 Batch 200 Loss 0.7375
Epoch 4 Batch 300 Loss 0.6737
Epoch 4 Loss 0.677926
Time taken for 1 epoch 1333.86314535141 sec

Epoch 5 Batch 0 Loss 0.6631
Epoch 5 Batch 100 Loss 0.6413


KeyboardInterrupt: 

In [ ]:
# Plot the losses
plt.plot(loss_plot)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Plot')
plt.show()

In [ ]:
# 9. Add the Caption on images

>The evaluate function is similar to the training loop, except you don't use teacher forcing here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.

>Stop predicting when the model predicts the end token and store the attention weights for every time step.

In [ ]:
def evaluate(image):
    attention_plot = np.zeros((max_length, attention_features_shape))

    hidden = decoder.reset_state(batch_size=1)

    temp_input = tf.expand_dims(load_image(image)[0], 0)
    img_tensor_val = image_features_extract_model(temp_input)
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[3]))

    features = encoder(img_tensor_val)

    dec_input = tf.expand_dims([tokenizer.word_index['<start>']], 0)
    result = []

    for i in range(max_length):
        predictions, hidden, attention_weights = decoder(dec_input, features, hidden)

        attention_plot[i] = tf.reshape(attention_weights, (-1, )).numpy()

        predicted_id = tf.random.categorical(predictions, 1)[0][0].numpy()
        result.append(tokenizer.index_word[predicted_id])

        if tokenizer.index_word[predicted_id] == '<end>':
            return result, attention_plot

        dec_input = tf.expand_dims([predicted_id], 0)

    attention_plot = attention_plot[:len(result), :]
    return result, attention_plot

In [ ]:
def plot_attention(image, result, attention_plot):
    temp_image = np.array(Image.open(image))

    fig = plt.figure(figsize=(10, 10))

    len_result = len(result)
    for l in range(len_result):
        temp_att = np.resize(attention_plot[l], (8, 8))
        ax = fig.add_subplot(len_result//2, len_result//2, l+1)
        ax.set_title(result[l])
        img = ax.imshow(temp_image)
        ax.imshow(temp_att, cmap='gray', alpha=0.6, extent=img.get_extent())

    plt.tight_layout()
    plt.show()

In [ ]:
# captions on the validation set
rid = np.random.randint(0, len(img_name_val))
image = img_name_val[rid]
real_caption = ' '.join([tokenizer.index_word[i] for i in cap_val[rid] if i not in [0]])
result, attention_plot = evaluate(image)

print ('Real Caption:', real_caption)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image, result, attention_plot)


# 10. Try it on any test images

In [ ]:
# Load my image
image_url = 'https://tensorflow.org/images/surf.jpg'

image_extension = image_url[-4:]
image_path = tf.keras.utils.get_file('image'+image_extension,
                                     origin=image_url)

result, attention_plot = evaluate(image_path)
print ('Prediction Caption:', ' '.join(result))
plot_attention(image_path, result, attention_plot)
# opening the image
Image.open(image_path)

##### Reference : https://www.tensorflow.org/tutorials/text/image_captioning